In [1]:
import tensorflow as tf
import numpy as np
tf.enable_eager_execution()
import importlib
import os

from transliteration import data, train, model_one, script, beam_search

In [2]:
importlib.reload(data)
batch_size = 128
train_dataset = data.make_dataset('../data/tfrecord/train.tfrecord',
                                  from_script='en',
                                  to_script='ja',
                                  batch_size=batch_size)
valid_dataset = data.make_dataset('../data/tfrecord/valid.tfrecord',
                                  from_script='en',
                                  to_script='ja',
                                  batch_size=batch_size)
test_dataset = data.make_dataset('../data/tfrecord/test.tfrecord',
                                 from_script='en',
                                 to_script='ja',
                                 batch_size=batch_size)

In [3]:
optimizer = tf.train.AdamOptimizer()


def loss_function(real, pred):
    mask = 1 - np.equal(real, 0)
    loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=real, logits=pred)
    return tf.reduce_mean(loss_ * mask)

In [19]:
importlib.reload(model_one)
encoder_config = model_one.Config(lstm_size=120,
                                  embedding_size=60,
                                  attention_size=30,
                                  vocab_size=script.SCRIPTS['en'].vocab_size)
decoder_config = model_one.Config(lstm_size=60,
                                  embedding_size=30,
                                  attention_size=30,
                                  vocab_size=script.SCRIPTS['ja'].vocab_size)
encoder = model_one.Encoder(encoder_config)
decoder = model_one.Decoder(decoder_config)

In [20]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [86]:
importlib.reload(train)
best_val_loss = None
for e in range(200):
    loss = train.run_one_epoch(train_dataset,
                               True,
                               from_script='en',
                               to_script='ja',
                               encoder=encoder,
                               decoder=decoder,
                               optimizer=optimizer,
                               loss_function=loss_function)
    valid_loss = train.run_one_epoch(valid_dataset,
                                     False,
                                     from_script='en',
                                     to_script='ja',
                                     encoder=encoder,
                                     decoder=decoder,
                                     loss_function=loss_function)
    if best_val_loss is None or valid_loss < best_val_loss:
        best_val_loss = valid_loss
        checkpoint.save(file_prefix=checkpoint_prefix)
    print("Epoch {}: Train Loss {:.3f}, Valid Loss {:.3f}".format(e, loss, valid_loss))
    print(train.transliterate(input_strs=['derick'],
                              from_script='en',
                              to_script='ja',
                              encoder=encoder,
                              decoder=decoder))

KeyboardInterrupt: 

In [105]:
importlib.reload(train)
importlib.reload(data)
importlib.reload(beam_search)
train.transliterate(input_strs=['derick'],
                    from_script='en',
                    to_script='ja',
                    encoder=encoder,
                    decoder=decoder,
                    # decoding_method=train.greedy_decode)
                    decoding_method=beam_search.beam_search_decode)

['デリック', 'デリクク', 'デリッキ', 'デリックルク', 'デリックリクク']


['デリック']

In [42]:
data.intern_katakana_char('<end>')

98